<a href="https://colab.research.google.com/github/sgun0315/Ask-Ernie-Big-Query/blob/main/Ask_Ernie_Update_Good_Known_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install google-api-core
%pip install --upgrade google-cloud-bigquery

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import vertexai

project_id = "syy-dom-np-fd67"
region = "us-central1"
vertexai.init(project=project_id, location=region)

In [ ]:
BQ_DATASET_NAME = "scm_chatbot_datastore_dev"
BQ_KNOWLEDGE_DATASET_NAME = "chatbot_knowledge_datastore_dev"
BQ_BUSINESS_KNOWLEDGE_TABLE_NAME = "business_rules_v2"

APPLICATION_NAME = "Merchandising"
GOOGLE_CLOUD_QUOTA_PROJECT = "syy-dom-np-fd67"
GOOGLE_CLOUD_QUOTA_PROJECT_REGION = "us-central1"

VECTOR_BUSINESS_RULES_MATCH_SCORE_MATCHES = 0.6

In [ ]:
import logging
from typing import Literal

from google.api_core.exceptions import GoogleAPICallError, BadRequest, ServiceUnavailable
from google.cloud import bigquery

logger = logging.getLogger(__name__)

class BQConnector:
    """Big Query Connector Implementation."""

    def __init__(self, source, project_id, region):
        self.source = source

        self.project_id = project_id
        self.region = region
        self.dataset_name = BQ_DATASET_NAME
        self.chatbot_knowledge_dataset = BQ_KNOWLEDGE_DATASET_NAME

        self._client = bigquery.Client(project=project_id, location=region)


    def get_all_business_rules(self) -> list[dict]:
      """Fetch all business rules with their content."""
      table_name = BQ_BUSINESS_KNOWLEDGE_TABLE_NAME
      full_table_path = f"{self.project_id}.{self.chatbot_knowledge_dataset}.{table_name}"

      query = f"""
          SELECT ruleset, content
          FROM `{full_table_path}`
          WHERE source_type = '{self.source}'
      """

      try:
          df = self.execute_query(query)
          logger.info("✅ Retrieved %d business rules from table %s", len(df), table_name)
          return df.to_dict(orient="records")
      except Exception as e:
          logger.error("❌ Failed to retrieve business rules: %s", str(e), exc_info=True)
          return []


    def execute_query(self, query):
        """Execute the given query and return the result in dataframe"""
        try:
            return self._client.query_and_wait(query).to_dataframe()
        except (TimeoutError, GoogleAPICallError, BadRequest, ServiceUnavailable) as e:
            logger.error("Error Executing Query: %s", e)
            raise ResourceConnectionError(f"Error executing query: {e}", service="BigQuery", errors=e) from e

In [ ]:
import logging
from typing import Optional, List, Literal

from vertexai.language_models import TextEmbeddingModel, TextEmbeddingInput

logger = logging.getLogger(__name__)


class EmbedderAgent:
    """Embedder Agent Implementation"""

    agent_type: str = "EmbedderAgent"

    def __init__(self, embeddings_model: str = "text-embedding-004"):
        self.embeddings_model = embeddings_model
        self.model = TextEmbeddingModel.from_pretrained(model_name=embeddings_model)
        logger.info("Embedder Agent Initialized in the given project.")

    async def create(
            self,
            questions,
            task: Literal["RETRIEVAL_QUERY", "RETRIEVAL_DOCUMENT", "SEMANTIC_SIMILARITY"] = "SEMANTIC_SIMILARITY",
            dimensionality: Optional[int] = 768,
    ) -> List[List[float]]:
        """Text embedding with a Large Language Model."""

        inputs = [TextEmbeddingInput(text, task) for text in questions]
        kwargs = {"output_dimensionality": dimensionality}
        embeddings = await self.model.get_embeddings_async(inputs, **kwargs)
        return [embedding.values for embedding in embeddings]

In [ ]:

async def get_embeddings_for_text(text):
    """Get Text Embeddings from Model"""
    embedder = EmbedderAgent(embeddings_model='text-embedding-004')
    embeddings = await embedder.create([text])
    return embeddings[0]

In [ ]:
bq_connector = BQConnector(
    APPLICATION_NAME,
    GOOGLE_CLOUD_QUOTA_PROJECT,
    GOOGLE_CLOUD_QUOTA_PROJECT_REGION
)

In [ ]:
import pandas as pd
rules_df = pd.DataFrame(
    {
        "ruleset": [
            "purchase order primary status should be 'OPEN' and purchase order secondary status should be 'ACCEPTED' and purchase order type should be 'REGULAR' and vendor id should not be equals to '359958' or '495149'. no need to consider about order control number or scheduled purchase order receipt timestamp",
            "(anticipated receipt date + given time interval or date) should be older than today date and purchase order primary status equals to 'OPEN' and purchase order secondary status not equals to 'PARTIALLY_RECEIVED'",
            "date of scheduled purchase order receipt timestamp should be before today's date. purchase order primary status should be 'OPEN' and purchase order secondary status should be 'SCHEDULED'",
            "order control number is null and purchase order primary status should be 'OPEN'",
            "anticipated receipt date should be before than today. purchase order type should be 'REGULAR' and purchase order primary status should be 'OPEN' and purchase order secondary status either 'SCHEDULED' OR 'ACCEPTED'",
            "purchase order primary status should be 'RECOMMENDED' and purchase order secondary status should be 'RECOMMENDED' and replenishment system should be 'DPR'",
            "purchase order primary status should be 'CLOSED'. If {threshold} is given in the question, then total vendor short quantity should be lesser than or equal to the negative value of the given {threshold}. If {threshold} is not given then compare with zero. {threshold} is a non negative value",
            "purchase order primary status should be 'CLOSED'. If the question includes a threshold, minimum value, then total vendor short quantity should be greater than or equal to that value. If no such value is mentioned compare with 0. threshold is a non negative value. Do not use TOTAL_RECEIVED_CASES or any other fields in the filtering logic"
        ],
        "content": [
            "Unscheduled PO or Unscheduled Purchase Order",
            "Purchase Orders or POs older than given month",
            "Purchase Order or PO with past scheduled appointment",
            "Purchase Order or PO without Order Control Number | Invalid PO | POs for cleanup",
            "Late PO or Late Purchase Order",
            "PO or Purchase Order with 'RECOMMENDED' status",
            "Purchase Order or POs with over of total quantity | overage POs",
            "Purchase Order or POs with short of total quantity. | shortage PO's."
        ],
        "source_type": [
            APPLICATION_NAME,
            APPLICATION_NAME,
            APPLICATION_NAME,
            APPLICATION_NAME,
            APPLICATION_NAME,
            APPLICATION_NAME,
            APPLICATION_NAME,
            APPLICATION_NAME
        ],
        "app": [
            "SIM360",
            "SIM360",
            "SIM360",
            "SIM360",
            "SIM360",
            "SIM360",
            "SIM360",
            "SIM360"
        ]
    }
)


In [ ]:
import asyncio

async def generate_and_upload_business_rule_embeddings():
    # Step 2: Generate embeddings
    embeddings = await asyncio.gather(*[get_embeddings_for_text(text) for text in rules_df["content"]])
    rules_df["embedding"] = [list(map(float, emb)) for emb in embeddings]  # Convert to list[float]

    # Step 3: Prepare rows
    records = []
    for _, row in rules_df.iterrows():
        records.append({
            "ruleset": row["ruleset"],
            "content": row["content"],
            "embedding": row["embedding"],
            "source_type": row["source_type"],
            "app": row["app"],
        })

    # Step 4: Upload to BigQuery
    table_path = f"{GOOGLE_CLOUD_QUOTA_PROJECT}.{BQ_KNOWLEDGE_DATASET_NAME}.{BQ_BUSINESS_KNOWLEDGE_TABLE_NAME}"
    errors = bq_connector._client.insert_rows_json(table_path, records)


    if errors:
        print("❌ Failed to insert rows:", errors)
    else:
        print("✅ Business rule embeddings uploaded successfully!")


In [ ]:
await generate_and_upload_business_rule_embeddings()

✅ Business rule embeddings uploaded successfully!


In [ ]:
bq_connector.get_all_business_rules()

[{'ruleset': "purchase order primary status should be 'OPEN' and purchase order secondary status should be 'ACCEPTED' and purchase order type should be 'REGULAR' and vendor id should not be equals to '359958' or '495149'. no need to consider about order control number or scheduled purchase order receipt timestamp",
  'content': 'Unscheduled PO or Unscheduled Purchase Order'},
 {'ruleset': "(anticipated receipt date + given time interval or date) should be older than today date and purchase order primary status equals to 'OPEN' and purchase order secondary status not equals to 'PARTIALLY_RECEIVED'",
  'content': 'Purchase Orders or POs older than given month'},
 {'ruleset': "date of scheduled purchase order receipt timestamp should be before today's date. purchase order primary status should be 'OPEN' and purchase order secondary status should be 'SCHEDULED'",
  'content': 'Purchase Order or PO with past scheduled appointment'},
 {'ruleset': "order control number is null and purchase or